# <center> Process WhatsApp messages from NDOH

## Read in WhatsApp messages

- Currently saved as **`.txt`** files in **```data/doh_whatsapp```**

In [1]:
# TODO : find out how these files are saved/ stored ? - Is it manual ?

## Import Libraries

In [747]:
import pandas as pd
from pathlib import Path
import regex
import string

In [622]:
# read data
data_path = Path("../data/doh_whatsapp/")
data_files = list(data_path.glob("*.txt"))

In [624]:
data_files =sorted(data_files)    # sort data , easier to debug - more intuitive 

## Create Regex Queries

In [699]:
kzn_query = r'(kwazulu[\s-–]*natal|kzn)[\s:,]+(\d{1,5})'    # use dash and or hyphen (different ascii representations)
kzn_query_1 = r'(kwazulu[\s-–]*natal|kzn)[ :,]*(\d{1,5})'    # use an actual space instead of \s because \s includes \r and \n
kzn_query_2 = r'(\d{1,5}[ :,]*kwazulu[\s-–]*natal|\d{1,5}kzn)'

gp_query = r'(gauteng)[\s:,]*(\d{1,5})'
gp_query_1 = r'(gauteng)[ :,]*(\d{1,5})'
gp_query_2 = r'(\d{1,5})[ :,]*(gauteng)'

wp_query = r'(Western Cape)[\s:,]*(\d{1,5})'
wp_query_1 = r'(Western Cape)[ :,]*(\d{1,5})'
wp_query_2 = r'(\d{1,5})[ :,]+(Western Cape)'

fs_query = r'(Free State)[\s:,]*(\d{1,5})'
fs_query_1 = r'(Free State)[ :,]*(\d{1,5})'
fs_query_2 = r'(\d{1,5})[ :,]*(Free State)'

nw_query = r'(North West)[\s:,]*(\d{1,5})'
nw_query_1 = r'(North West)[ :,]*(\d{1,5})'
nw_query_2 = r'(\d{1,5})[ :,]*(North West)'

mp_query = r'(Mpumalanga)[\s:,]*(\d{1,5})'
mp_query_1 = r'(Mpumalanga){s<=2:[a-z]}[ :,]*(\d{1,5})'    # to save against misspellings we allow at most 2 substitutions from [a-z]
mp_query_2 = r'(\d{1,5})[ :,]*(Mpumalanga){s<=2:[a-z]}'

limp_query = r'(Limpopo)[\s:,]*(\d{1,5})'
limp_query_1 = r'(Limpopo)[ :,]*(\d{1,5})'
limp_query_2 = r'(\d{1,5})[ :,]*(Limpopo)'

ec_query = r'(Eastern Cape)[\s:,]*(\d{1,5})'
ec_query_1 = r'(Eastern Cape)[ :,]*(\d{1,5})'
ec_query_2 = r'(\d{1,5})[ :,]*(Eastern Cape)'

nc_query = r'(Northern Cape)[\s:,]*(\d{1,5})'
nc_query_1 = r'(Northern Cape)[ :,]*(\d{1,5})'
nc_query_2 = r'(\d{1,5})[ :,]*(Northern Cape)'

unk_query = r'(UNALLOCATED|Unknown)[\s:,]*(\d{1,5})'
unk_query_1 = r'(UNALLOCATED|Unknown)[ :,]*(\d{1,5})'
unk_query_2 = r'(\d{1,5})[ :,]*(UNALLOCATED|Unknown)'

In [908]:
# TODO : limit search to just the number breakdown portion of message
query_dict = {'KZN' : [kzn_query_1, kzn_query_2],
              'Gauteng' : [gp_query_1, gp_query_2],
              'Western Cape' : [wp_query_1, wp_query_2], 
              'Free State' : [fs_query_1, fs_query_2],
              'North West' : [nw_query_1, nw_query_2], 
              'Mpumalanga' : [mp_query_1, mp_query_2],
              'Limpopo' : [limp_query_1, limp_query_2],
              'Eastern Cape' : [ec_query_1, ec_query_2],
              'Northern Cape' : [nc_query_1, nc_query_2],
              'Unknown' : [unk_query_1, unk_query_2]}

# TODO : make this loopy code more effecient
dates = []
provinces = []
counts = []

for i, data_file in enumerate(data_files):
    date = data_file.stem.split("-whatsapp-cases")[0]    # we assume that the naming convention for the stored text file is consistent
    print('[',i+1,']', date) 
         
    match_counter = 0
    
    with open(data_file,'r') as reader :
        data = reader.readlines()
        
    data_string = " ".join([d for d in data])    # convert data to a single string, could maybe use re.MULTILINE instead ?
        
    for prov, prov_query in query_dict.items():
        for query in prov_query:
            match = regex.search(query, data_string, re.IGNORECASE)
            if match:
                m = match.group()
                count = "".join([s for s in m if s.isdigit()])   # strip infection count from string
                
                counts.append(int(count))
                provinces.append(prov)
                dates.append(date1)
                
                match_counter+=1

# 


[ 1 ] 2020-03-16
[ 2 ] 2020-03-17
[ 3 ] 2020-03-20
[ 4 ] 2020-03-23
[ 5 ] 2020-03-24
[ 6 ] 2020-03-25
[ 7 ] 2020-03-26
[ 8 ] 2020-03-27
[ 9 ] 2020-03-28
[ 10 ] 2020-03-29
[ 11 ] 2020-03-30
[ 12 ] 2020-03-31
[ 13 ] 2020-04-02
[ 14 ] 2020-04-03
[ 15 ] 2020-04-04
[ 16 ] 2020-04-05
[ 17 ] 2020-04-08


In [909]:
df = pd.DataFrame({'Date':dates,
                   'Province':provinces,
                   'Infection Count':counts})

In [895]:
by_date = df.groupby('Date')
by_province = df.groupby('Province')

In [896]:
by_province.get_group('KZN')

,Date,Province,Infection Count
0,2020-03-16,KZN,12
5,2020-03-17,KZN,16
10,2020-03-20,KZN,23
16,2020-03-23,KZN,60
26,2020-03-24,KZN,80
35,2020-03-25,KZN,91
45,2020-03-26,KZN,134
55,2020-03-27,KZN,134
65,2020-03-28,KZN,156
75,2020-03-29,KZN,167


In [884]:
# test all queries combined
# query_list = [{'kzn' : kzn_query_1, kzn_query_2},
#               {'gp' : gp_query_1, gp_query_2},
#               {'wp' : wp_query_1, wp_query_2}, 
#               {'fs' : fs_query_1, fs_query_2},
#               {'nw' : nw_query_1, nw_query_2}, 
#               {'mp' : mp_query_1, mp_query_2},
#               {'limp' : limp_query_1, limp_query_2},
#               {'ec' : ec_query_1, ec_query_2},
#               {'nc' : nc_query_1, nc_query_2},
#               {'unk' : unk_query_1, unk_query_2}]

# # loop over all files
# # TODO : limit search to just the number breakdown portion of message

# dates = []
# provinces = []
# counts = []

# for i, data_file in enumerate(data_files):
#     date1 = data_file.stem.split("-whatsapp-cases")[0]    # we assume that the naming convention for the stored text file is consistent
#     print('[',i+1,']', date1) 
         
#     match_counter = 0
    
#     with open(data_file,'r') as reader :
#         data = reader.readlines()
        
#     data_string = " ".join([d for d in data])    # convert data to a single string, could maybe use re.MULTILINE instead ?
    
#     for i,q in enumerate(query_list):
#         match = regex.search(q, data_string, re.IGNORECASE)
#         if match:
#             m = match.group()
# #             print(f"query {i+1} of  {len(query_list)} : {m}")
#             match_counter+=1
       
#             m = m.replace(":"," ").split()
            
#             counts.append([int(s) for s in m if s.isdigit()][0])
#             provs.append(" ".join([s for s in m if not s.isdigit()]))
#             dates.append(date1)
# #             proc_data.append((date1,prov, counts[0]))
    
# #         else:
# #             print(f"no match for query {i+1} of {len(query_list)} ")
            
# #     print(f"{match_counter} matches found")
# #     print("--------"*10) 
    

In [882]:
# a=list(zip(dates,provs,counts))
# for t in a:
#     print(t[1])

In [883]:
# ## get date for each file
# # -whatsapp-cases
# dates = [data_file.stem.split("-whatsapp-cases")[0] for i,data_file in enumerate(data_files)]
# dates


In [632]:
# heading = data[0]
# date = data[1]
# summary = data[2:8]
# stats_breakdown = data[9:20]
# end_text = data[21:]

In [633]:
# print(f"heading : {heading}")
# print("---"*25)

# print(f"date : {date}")
# print("---"*25)

# print(f"summary : {[print(s) for s in summary]}")
# print("---"*25)

# print(f"stats_breakdown : {[print(stats) for stats in stats_breakdown]}")
# print("---"*25)

# print(f"end_text : {end_text}")
# print("---"*25)


In [634]:
# date.strip().split('-')